<a href="https://colab.research.google.com/github/acordobaf/tpp/blob/main/simulacion_tesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from IPython.display import display


# Lista de personal disponible en gamelas
personal_gamelas = {
    "Blanquita": 35,
    "La Irene": 18,
    "La Carolina": 0,
    "La Costa": 0,
    "El Mirasol": 0,
    "San Victorio": 18,
    "La Esperanza": 10,
}
personal_gamelas_df = pd.DataFrame(list(personal_gamelas.items()), columns=['Chacra', 'Disponible gamelas'])

# Lista de personal local
personal_local = {
    "Blanquita": 0,
    "La Irene": 2,
    "La Carolina": 4,
    "La Costa": 5,
    "El Mirasol": 4,
    "San Victorio": 1,
    "La Esperanza": 4,
}
personal_local_df = pd.DataFrame(list(personal_local.items()), columns=['Chacra', 'Disponible local'])

#Disponibilidad de personal, sin considerar alquileres
data = {
    "Chacra": personal_gamelas.keys(),
    "Personal Gamelas": personal_gamelas.values(),
    "Personal Local": personal_local.values(),
}
personal_disponible_df = pd.DataFrame(data)

# Lista de costos mensuales de alquiler por chacra
rental_cost = {
    "Blanquita": 0,
    "La Irene": 600,
    "La Carolina": 600,
    "La Costa": 2000,
    "El Mirasol": 0,
    "San Victorio": 0,
    "La Esperanza": 1600,
}

# Lista de capacidad de los alquileres
personal_alquileres= {
    "Blanquita": 0,
    "La Irene": 6,
    "La Carolina": 6,
    "La Costa": 12,
    "El Mirasol": 0,
    "San Victorio": 0,
    "La Esperanza": 10,
}

personal_alquileres_df = pd.DataFrame(list(personal_alquileres.items()), columns=['Chacra', 'Disponible alquiler'])

# Necesidades de personal para las 18 semanas completas
needs_extended = {
    "Semana 2": [15, 14, 3, 4, 8, 1, 10],
    "Semana 3": [15, 14, 3, 4, 8, 1, 10],
    "Semana 4": [45, 42, 8, 12, 24, 4, 31],
    "Semana 5": [34, 32, 6, 9, 19, 3, 24],
    "Semana 6": [31, 13, 12, 34, 22, 0, 10],
    "Semana 7": [24, 17, 12, 23, 30, 0, 18],
    "Semana 8": [24, 16, 12, 23, 28, 3, 17],
    "Semana 9": [11, 0, 0, 45, 1, 6, 16],
    "Semana 10": [19, 0, 0, 11, 1, 10, 13],
    "Semana 11": [8, 0, 0, 4, 1, 4, 3],
    "Semana 12": [0, 0, 0, 0, 0, 0, 0],
    "Semana 13": [0, 0, 3, 34, 0, 0, 1],
    "Semana 14": [0, 0, 4, 51, 0, 0, 1],
    "Semana 15": [0, 0, 4, 41, 0, 0, 1],
    "Semana 16": [0, 0, 5, 54, 0, 0, 1],
    "Semana 17": [0, 0, 5, 57, 0, 0, 1],
    "Semana 18": [0, 0, 2, 25, 0, 0, 1],
}

# Convertir las necesidades a DataFrame
needs_extended_df = pd.DataFrame(needs_extended)

# Generar las proyecciones con variabilidad
projected_needs_extended = []

# Año 1: Usar los valores originales
projected_needs_extended.append({"Año": "Año 1", **needs_extended})

# Años 2 al 10: Aplicar variabilidad aleatoria
for year in range(2, 11):
    variability = np.random.uniform(0.90, 1.10, size=needs_extended_df.shape)
    needs_with_variability = needs_extended_df * variability  # Aplicar variabilidad
    year_data = {"Año": f"Año {year}"}
    for week in needs_with_variability.columns:
        year_data[week] = needs_with_variability[week].values.tolist()
    projected_needs_extended.append(year_data)

# Convertir las proyecciones a DataFrame final
projected_needs_extended_df = pd.DataFrame(projected_needs_extended)
display(projected_needs_extended_df)





#Comparacion de costos por chacra, semana y año

exploded_results_extended = []

# Iterar sobre las proyecciones extendidas
for _, row in projected_needs_extended_df.iterrows():


    year = row["Año"]

    daily_wage = 50 # Costo de jornal por persona por día
    work_days = 5.5 # Días de trabajo por semana

    mobile_housing_capacities = [10, 15, 20, 30] # Capacidades de viviendas móviles
    mobile_housing_costs = [1500, 2000, 2500, 3000] # Costo mensual por unidad de vivienda móvil
    monthly_to_weekly_factor = 4  # Prorratear mensual a semanal
    mobile_cost = 0
    mobile_capacity_used = 0

    construction_costs = [40000, 50000, 60000] # Costos de construcción por casa
    construction_capacities = [20, 25, 30] # Capacidades de construcción de casas
    weeks_in_5_years = 260  # Total de semanas en 5 años
    construction_cost = 0
    construction_capacity_used = 0

    sprinter_cost = 420 # Costo por viaje de Sprinter (18 asientos)
    sprinter_capacity = 18
    sprinter_total_cost = 0
    bus_cost = 1000  # Costo por viaje de Bus (50 asientos)
    bus_capacity = 50
    bus_total_cost = 0
    transport_cost = 0
    transport_option = "-"

    rental_cost_weekly = 0
    monthly_to_weekly_factor = 4
    alquiler_estado = {chacra: 0 for chacra in personal_alquileres_df["Chacra"]}

    for week in needs_extended.keys():
        weekly_needs = row[week]  # Lista de necesidades por chacra

        for chacra_idx, need in enumerate(weekly_needs):
            chacra = personal_disponible_df.iloc[chacra_idx]['Chacra']
            available_personnel = personal_disponible_df.loc[chacra_idx, "Personal Gamelas"] + personal_disponible_df.loc[chacra_idx, "Personal Local"]
            balance = available_personnel - need

            if alquiler_estado != 0: alquiler_estado[chacra] -= 1

            if balance < 0 and alquiler_estado == 0:  # Déficit

                  mobile_capacity_used = 0
                  construction_capacity_used = 0
                  deficit = abs(balance)
                  transport_cost = 0
                  transport_option = ""


                  rental_cost_weekly = rental_cost.get(chacra, 0) / monthly_to_weekly_factor


                  # Calcular costos de transporte considerando Sprinter y Bus
                  sprinter_trips = -(-deficit // sprinter_capacity)  # Redondeo hacia arriba
                  sprinter_total_cost = sprinter_trips * sprinter_cost * work_days

                  bus_trips = -(-deficit // bus_capacity)  # Redondeo hacia arriba
                  bus_total_cost = bus_trips * bus_cost * work_days


                  if sprinter_total_cost <= bus_total_cost:
                      transport_cost = sprinter_total_cost
                      transport_option = "Sprinter"
                  else:
                      transport_cost = bus_total_cost
                      transport_option = "Bus"



                  # Costo de viviendas móviles
                  mobile_costs = [
                      (capacity, mobile_housing_cost)
                      for mobile_housing_cost in mobile_housing_costs
                      for capacity in mobile_housing_capacities

                  ]

                  best_mobile_option = min(mobile_costs, key=lambda x: x[0], default=(0, float("inf")))

                  if best_mobile_option[0] > 0:  # Si existe una opción válida
                      mobile_cost = best_mobile_option[1] / 4  # Prorratear a semanal
                      mobile_capacity_used = best_mobile_option[0]
                  else:
                      mobile_cost = float("inf")
                      mobile_capacity_used = 0


                # Costo de construcción
                  construction_costs_with_capacity = [
                      (capacity, cost)
                      for cost in construction_costs
                      for capacity in construction_capacities

                  ]

                  best_construction_option = min(construction_costs_with_capacity, key=lambda x: x[0], default=(0, float("inf")))

                  if best_construction_option[0] > 0:  # Si existe una opción válida
                      construction_capacity_used = best_construction_option[0]  # Capacidad utilizada
                      construction_cost = best_construction_option[1] / weeks_in_5_years  # Costo prorrateado semanal
                  else:
                      construction_capacity_used = 0
                      construction_cost = float("inf")  # No hay opción válida


            else:  # Superávit
                surplus = balance
                surplus_cost = surplus * daily_wage * work_days #costo de mantener la gente en la chacra


            # Elegir la mejor opción en función del costo total

            costs = {
                f"Transporte ({transport_option}) ($)": transport_cost,
                "Alquiler ($)": rental_cost_weekly,
                "Vivienda móvil ($)": mobile_cost,
                "Construcción ($)": construction_cost/weeks_in_5_years,
                "Mantener Superávit ($)": surplus_cost
            }


            best_option = min(costs, key=costs.get)
            best_cost = costs[best_option]
            capacity_used = 0

            # Si se elige alquiler, activarlo por 4 semanas
            if best_option == "Alquiler ($)":
                alquiler_estado[chacra] = 4
                best_option = "Alquiler ($)"
                best_cost = rental_cost_weekly

            # Registrar resultados ajustados
            exploded_results_extended.append({
                "Año": year,
                "Semana": week,
                "Chacra": chacra,
                "Balance": balance,
                "costo de mantener gente ahi": surplus_cost,
                "costo de moverlos en sprinter": sprinter_total_cost,
                "costo de moverlos en bus": bus_total_cost,
                "costo de alquiler": rental_cost_weekly,
                "Capacidad Vivienda Móvil": mobile_capacity_used,
                "costo de alquilar una vivienda movil": mobile_cost,
                "Capacidad Construcción": construction_capacity_used,
                "costo de construir una casa": construction_cost,
                "Costo semanal": best_cost,
                "Opción económica": best_option,

            })

# Convertir resultados a DataFrame
exploded_results_extended_df = pd.DataFrame(exploded_results_extended)

# Mostrar resultados finales por las 18 semanas, chacra y año

display(exploded_results_extended_df.round(0))


display(exploded_results_extended_df.round(0).groupby(["Año", "Semana", "Chacra"]).agg({
    "Balance": "sum",
    "Costo semanal": "sum"
}))


# Guardar el DataFrame en un archivo .xlsx
exploded_results_extended_df.to_excel("Optimización_Completa.xlsx", index=False)

# Opción para descargar el archivo desde Google Colab
from google.colab import files
files.download("Optimización_Completa.xlsx")


In [ ]:
import numpy as np
import pandas as pd
from IPython.display import display

# Datos iniciales
personal_gamelas = {
    "Blanquita": 35,
    "La Irene": 18,
    "La Carolina": 0,
    "La Costa": 0,
    "El Mirasol": 0,
    "San Victorio": 18,
    "La Esperanza": 10,
}

personal_local = {
    "Blanquita": 0,
    "La Irene": 2,
    "La Carolina": 4,
    "La Costa": 5,
    "El Mirasol": 4,
    "San Victorio": 1,
    "La Esperanza": 4,
}

personal_alquileres = {
    "Blanquita": 0,
    "La Irene": 60,
    "La Carolina": 60,
    "La Costa": 120,
    "El Mirasol": 0,
    "San Victorio": 0,
    "La Esperanza": 100,
}

rental_cost = {
    "Blanquita": 0,
    "La Irene": 600,
    "La Carolina": 600,
    "La Costa": 2000,
    "El Mirasol": 0,
    "San Victorio": 0,
    "La Esperanza": 1600,
}

needs_extended = {
    "Semana 2": [15, 14, 3, 4, 8, 1, 10],
    "Semana 3": [15, 14, 3, 4, 8, 1, 10],
    "Semana 4": [45, 42, 8, 12, 24, 4, 31],
    "Semana 5": [34, 32, 6, 9, 19, 3, 24],
    "Semana 6": [31, 13, 12, 34, 22, 0, 10],
    "Semana 7": [24, 17, 12, 23, 30, 0, 18],
    "Semana 8": [24, 16, 12, 23, 28, 3, 17],
    "Semana 9": [11, 0, 0, 45, 1, 6, 16],
    "Semana 10": [19, 0, 0, 11, 1, 10, 13],
    "Semana 11": [8, 0, 0, 4, 1, 4, 3],
    "Semana 12": [0, 0, 0, 0, 0, 0, 0],
    "Semana 13": [0, 0, 3, 34, 0, 0, 1],
    "Semana 14": [0, 0, 4, 51, 0, 0, 1],
    "Semana 15": [0, 0, 4, 41, 0, 0, 1],
    "Semana 16": [0, 0, 5, 54, 0, 0, 1],
    "Semana 17": [0, 0, 5, 57, 0, 0, 1],
    "Semana 18": [0, 0, 2, 25, 0, 0, 1],
}

# Crear DataFrames
personal_gamelas_df = pd.DataFrame(list(personal_gamelas.items()), columns=['Chacra', 'Disponible Gamelas'])
personal_local_df = pd.DataFrame(list(personal_local.items()), columns=['Chacra', 'Disponible Local'])
personal_alquileres_df = pd.DataFrame(list(personal_alquileres.items()), columns=['Chacra', 'Capacidad Alquiler'])
needs_extended_df = pd.DataFrame(needs_extended)

# Combinamos disponibilidad total
personal_disponible_df = pd.DataFrame({
    'Chacra': personal_gamelas.keys(),
    'Disponible Total': [g + l for g, l in zip(personal_gamelas.values(), personal_local.values())],
})

# Proyecciones con variabilidad
projected_needs_extended = []
projected_needs_extended.append({"Año": "Año 1", **needs_extended})
for year in range(2, 11):
    variability = np.random.uniform(0.90, 1.10, size=needs_extended_df.shape)
    needs_with_variability = (needs_extended_df * variability).round(0)
    year_data = {"Año": f"Año {year}"}
    for week in needs_with_variability.columns:
        year_data[week] = needs_with_variability[week].values.tolist()
    projected_needs_extended.append(year_data)

projected_needs_extended_df = pd.DataFrame(projected_needs_extended)

# Función para calcular costos

def calcular_costos(chacra, balance, work_days, daily_wage):
    sprinter_cost, sprinter_capacity = 420, 18
    bus_cost, bus_capacity = 1000, 50

    if balance < 0:  # Déficit: calcular transporte y otros costos

        surplus_cost = float('inf')

        deficit = abs(balance)
        transport_cost_sprinter = -(-deficit // sprinter_capacity) * sprinter_cost * work_days
        transport_cost_bus = -(-deficit // bus_capacity) * bus_cost * work_days

        mobile_housing_costs = [1500, 2000, 2500, 3000]
        mobile_housing_capacities = [10, 15, 20, 30]

        best_mobile_option = min(
            [(c, cost) for c, cost in zip(mobile_housing_capacities, mobile_housing_costs) if c >= deficit],
            default=(0, float('inf')),
            key=lambda x: x[1],
        )


        rental_capacity = personal_alquileres_df.loc[personal_alquileres_df['Chacra'] == chacra, 'Capacidad Alquiler'].values[0]
        rental_weekly_cost = rental_cost.get(chacra, 0) / 4 if rental_capacity >= deficit else float('inf')

        return {
            "Sprinter": transport_cost_sprinter,
            "Bus": transport_cost_bus,
            "Móvil": best_mobile_option[1] / 4,
            "Alquiler": rental_weekly_cost,
            "Ociosidad": surplus_cost,
        }

    elif balance > 0:  # Superávit: costo de personal extra

        surplus_cost = balance * daily_wage * work_days

        deficit = abs(balance)
        transport_cost_sprinter = -(-deficit // sprinter_capacity) * sprinter_cost * work_days
        transport_cost_bus = -(-deficit // bus_capacity) * bus_cost * work_days

        mobile_housing_costs = [1500, 2000, 2500, 3000]
        mobile_housing_capacities = [10, 15, 20, 30]

        best_mobile_option = min(
            [(c, cost) for c, cost in zip(mobile_housing_capacities, mobile_housing_costs) if c >= deficit],
            default=(0, float('inf')),
            key=lambda x: x[1],
        )

        rental_capacity = personal_alquileres_df.loc[personal_alquileres_df['Chacra'] == chacra, 'Capacidad Alquiler'].values[0]
        rental_weekly_cost = rental_cost.get(chacra, 0) / 4 if rental_capacity >= deficit else float('inf')

        return {
            "Sprinter": transport_cost_sprinter,
            "Bus": transport_cost_bus,
            "Móvil": best_mobile_option[1] / 4,
            "Alquiler": rental_weekly_cost,
            "Ociosidad": surplus_cost,
        }
    else:  # No hay necesidad ni superávit
        return {key: 0 for key in ["Sprinter", "Bus", "Móvil", "Construcción", "Alquiler", "Ociosidad"]}

# Función para calcular inversiones

def calcular_inversiones(deficit):
    construction_costs = [40000, 50000, 60000]
    construction_capacities = [20, 25, 30]
    best_construction_option = min(
        [(c, cost) for c, cost in zip(construction_capacities, construction_costs) if c >= deficit],
        default=(0, float('inf')),
        key=lambda x: x[1],
    )
    return best_construction_option




# Iterar sobre proyecciones
exploded_results = []
inversiones = []
alquiler_estado = {chacra: 0 for chacra in personal_disponible_df['Chacra']}
alquiler_contador = {chacra: 0 for chacra in personal_disponible_df['Chacra']}
current_investments = {}
amortizacion_aplicada = {chacra: False for chacra in personal_disponible_df['Chacra']}

daily_wage = 50  # Costo diario por persona
work_days = 5.5  # Días laborables por semana


for _, row in projected_needs_extended_df.iterrows():
    year = row['Año']

    # Apply amortization yearly for each chacra
    for chacra in personal_disponible_df['Chacra']:
        if not amortizacion_aplicada[chacra] and chacra in current_investments:
            inversion = current_investments[chacra]
            if int(year.split()[1]) - int(inversion['Año construcción'].split()[1]) < 10:
                amortizacion = inversion['Amortización']
                # Apply amortization to the first week of the year for this chacra
                # This might involve adjusting a cost variable or calculation for that week
                # Example: total_yearly_cost[chacra] -= amortizacion
            amortizacion_aplicada[chacra] = True  # Mark as applied for the year



    for week, needs in row.items():
            if week == 'Año':
                continue

            for chacra, need in zip(personal_disponible_df['Chacra'], needs):
                available = personal_disponible_df.loc[personal_disponible_df['Chacra'] == chacra, 'Disponible Total'].values[0]
                balance = available - need

                costos = calcular_costos(chacra, balance, work_days, daily_wage)
                best_option = min(costos, key=costos.get)
                best_cost = costos[best_option]

                if best_option == "Alquiler":
                    alquiler_contador[chacra] += 1
                    alquiler_estado[chacra] = 4
                else:
                    alquiler_contador[chacra] = 0
                    alquiler_contador[chacra] = 0

                if alquiler_contador[chacra] == 10:  # Si se alquila por 10 semanas consecutivas
                    deficit = abs(balance)
                    construction_capacity, construction_cost = calcular_inversiones(deficit)
                    if construction_capacity > 0:
                        current_investments[chacra] = {
                            "Año construcción": year,
                            "Costo": construction_cost,
                            "Amortización": construction_cost / 10,
                            "Semana construcción": week,
                        }
                        best_option = "Construcción"
                        best_cost = construction_cost

                if best_cost == 0:
                    best_option = "N/A"  # Asignar 'N/A' cuando el costo es cero



                exploded_results.append({
                    "Año": year,
                    "Semana": week,
                    "Chacra": chacra,
                    "Balance": balance,
                    "Ociosidad": costos["Ociosidad"],
                    "Sprinter": costos["Sprinter"],
                    "Bus": costos["Bus"],
                    "Vivienda Móvil": costos["Móvil"],
                    "Alquiler": costos["Alquiler"],
                    "Opcion": best_option,
                    "Costo Total": best_cost,
                    "Inversiones": amortizacion,
                })

    amortizacion_aplicada = {chacra: False for chacra in personal_disponible_df['Chacra']}

# Convertir a DataFrame final
exploded_results_df = pd.DataFrame(exploded_results)

# Guardar y mostrar resultados
exploded_results_df.to_excel("Optimizacion_Logistica.xlsx", index=False)
display(exploded_results_df)

from google.colab import files
files.download("Optimizacion_Logistica.xlsx")


,Año,Semana,Chacra,Balance,Ociosidad,Sprinter,Bus,Vivienda Móvil,Alquiler,Opcion,Costo Total,Inversiones
0,Año 1,Semana 2,Blanquita,20.0,5500.0,4620.0,5500.0,625.0,inf,Móvil,625.0,0.0
1,Año 1,Semana 2,La Irene,6.0,1650.0,2310.0,5500.0,375.0,150.0,Alquiler,150.0,0.0
2,Año 1,Semana 2,La Carolina,1.0,275.0,2310.0,5500.0,375.0,150.0,Alquiler,150.0,0.0
3,Año 1,Semana 2,La Costa,1.0,275.0,2310.0,5500.0,375.0,500.0,Ociosidad,275.0,0.0
4,Año 1,Semana 2,El Mirasol,-4.0,inf,2310.0,5500.0,375.0,inf,Móvil,375.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1185,Año 10,Semana 18,La Carolina,2.0,550.0,2310.0,5500.0,375.0,150.0,Alquiler,150.0,4000.0
1186,Año 10,Semana 18,La Costa,-18.0,inf,2310.0,5500.0,625.0,500.0,Alquiler,500.0,4000.0
1187,Año 10,Semana 18,El Mirasol,4.0,1100.0,2310.0,5500.0,375.0,inf,Móvil,375.0,4000.0
1188,Año 10,Semana 18,San Victorio,19.0,5225.0,4620.0,5500.0,625.0,inf,Móvil,625.0,4000.0


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import numpy as np
import pandas as pd
from IPython.display import display

# Datos iniciales
personal_gamelas = {
    "Blanquita": 35,
    "La Irene": 18,
    "La Carolina": 0,
    "La Costa": 0,
    "El Mirasol": 0,
    "San Victorio": 18,
    "La Esperanza": 10,
}

personal_local = {
    "Blanquita": 0,
    "La Irene": 2,
    "La Carolina": 4,
    "La Costa": 5,
    "El Mirasol": 4,
    "San Victorio": 1,
    "La Esperanza": 4,
}

personal_alquileres = {
    "Blanquita": 0,
    "La Irene": 60,
    "La Carolina": 60,
    "La Costa": 120,
    "El Mirasol": 0,
    "San Victorio": 0,
    "La Esperanza": 100,
}

rental_cost = {
    "Blanquita": 0,
    "La Irene": 600,
    "La Carolina": 600,
    "La Costa": 2000,
    "El Mirasol": 0,
    "San Victorio": 0,
    "La Esperanza": 1600,
}

needs_extended = {
    "Semana 2": [15, 14, 3, 4, 8, 1, 10],
    "Semana 3": [15, 14, 3, 4, 8, 1, 10],
    "Semana 4": [45, 42, 8, 12, 24, 4, 31],
    "Semana 5": [34, 32, 6, 9, 19, 3, 24],
    "Semana 6": [31, 13, 12, 34, 22, 0, 10],
    "Semana 7": [24, 17, 12, 23, 30, 0, 18],
    "Semana 8": [24, 16, 12, 23, 28, 3, 17],
    "Semana 9": [11, 0, 0, 45, 1, 6, 16],
    "Semana 10": [19, 0, 0, 11, 1, 10, 13],
    "Semana 11": [8, 0, 0, 4, 1, 4, 3],
    "Semana 12": [0, 0, 0, 0, 0, 0, 0],
    "Semana 13": [0, 0, 3, 34, 0, 0, 1],
    "Semana 14": [0, 0, 4, 51, 0, 0, 1],
    "Semana 15": [0, 0, 4, 41, 0, 0, 1],
    "Semana 16": [0, 0, 5, 54, 0, 0, 1],
    "Semana 17": [0, 0, 5, 57, 0, 0, 1],
    "Semana 18": [0, 0, 2, 25, 0, 0, 1],
}

# Crear DataFrames
personal_gamelas_df = pd.DataFrame(list(personal_gamelas.items()), columns=['Chacra', 'Disponible Gamelas'])
personal_local_df = pd.DataFrame(list(personal_local.items()), columns=['Chacra', 'Disponible Local'])
personal_alquileres_df = pd.DataFrame(list(personal_alquileres.items()), columns=['Chacra', 'Capacidad Alquiler'])
needs_extended_df = pd.DataFrame(needs_extended)

# Combinamos disponibilidad total
personal_disponible_df = pd.DataFrame({
    'Chacra': personal_gamelas.keys(),
    'Disponible Total': [g + l for g, l in zip(personal_gamelas.values(), personal_local.values())],
})

# Proyecciones con variabilidad
projected_needs_extended = []
projected_needs_extended.append({"Año": "Año 1", **needs_extended})
for year in range(2, 11):
    variability = np.random.uniform(0.90, 1.10, size=needs_extended_df.shape)
    needs_with_variability = (needs_extended_df * variability).round(0)
    year_data = {"Año": f"Año {year}"}
    for week in needs_with_variability.columns:
        year_data[week] = needs_with_variability[week].values.tolist()
    projected_needs_extended.append(year_data)

projected_needs_extended_df = pd.DataFrame(projected_needs_extended)

# Función para calcular costos

def calcular_costos(chacra, balance, work_days, daily_wage):
    sprinter_cost, sprinter_capacity = 420, 18
    bus_cost, bus_capacity = 1000, 50

    if balance < 0:  # Déficit: calcular transporte y otros costos

        surplus_cost = float('inf')

        deficit = abs(balance)
        transport_cost_sprinter = -(-deficit // sprinter_capacity) * sprinter_cost * work_days
        transport_cost_bus = -(-deficit // bus_capacity) * bus_cost * work_days

        mobile_housing_costs = [1500, 2000, 2500, 3000]
        mobile_housing_capacities = [10, 15, 20, 30]

        best_mobile_option = min(
            [(c, cost) for c, cost in zip(mobile_housing_capacities, mobile_housing_costs) if c >= deficit],
            default=(0, float('inf')),
            key=lambda x: x[1],
        )


        rental_capacity = personal_alquileres_df.loc[personal_alquileres_df['Chacra'] == chacra, 'Capacidad Alquiler'].values[0]
        rental_weekly_cost = rental_cost.get(chacra, 0) / 4 if rental_capacity >= deficit else float('inf')

        return {
            "Sprinter": transport_cost_sprinter,
            "Bus": transport_cost_bus,
            "Móvil": best_mobile_option[1] / 4,
            "Alquiler": rental_weekly_cost,
            "Ociosidad": surplus_cost,
        }

    elif balance > 0:  # Superávit: costo de personal extra

        surplus_cost = balance * daily_wage * work_days

        deficit = abs(balance)
        transport_cost_sprinter = -(-deficit // sprinter_capacity) * sprinter_cost * work_days
        transport_cost_bus = -(-deficit // bus_capacity) * bus_cost * work_days

        mobile_housing_costs = [1500, 2000, 2500, 3000]
        mobile_housing_capacities = [10, 15, 20, 30]

        best_mobile_option = min(
            [(c, cost) for c, cost in zip(mobile_housing_capacities, mobile_housing_costs) if c >= deficit],
            default=(0, float('inf')),
            key=lambda x: x[1],
        )

        rental_capacity = personal_alquileres_df.loc[personal_alquileres_df['Chacra'] == chacra, 'Capacidad Alquiler'].values[0]
        rental_weekly_cost = rental_cost.get(chacra, 0) / 4 if rental_capacity >= deficit else float('inf')

        return {
            "Sprinter": transport_cost_sprinter,
            "Bus": transport_cost_bus,
            "Móvil": best_mobile_option[1] / 4,
            "Alquiler": rental_weekly_cost,
            "Ociosidad": surplus_cost,
        }
    else:  # No hay necesidad ni superávit
        return {key: 0 for key in ["Sprinter", "Bus", "Móvil", "Construcción", "Alquiler", "Ociosidad"]}

# Iterar sobre proyecciones
exploded_results = []
inversiones = []
alquiler_estado = {chacra: 0 for chacra in personal_disponible_df['Chacra']}
alquiler_contador = {chacra: 0 for chacra in personal_disponible_df['Chacra']}
current_investments = {}
amortizacion_aplicada = {chacra: False for chacra in personal_disponible_df['Chacra']}

daily_wage = 50  # Costo diario por persona
work_days = 5.5  # Días laborables por semana


for _, row in projected_needs_extended_df.iterrows():
    year = row['Año']


    for week, needs in row.items():
            if week == 'Año':
                continue

            for chacra, need in zip(personal_disponible_df['Chacra'], needs):
                available = personal_disponible_df.loc[personal_disponible_df['Chacra'] == chacra, 'Disponible Total'].values[0]
                balance = available - need

                costos = calcular_costos(chacra, balance, work_days, daily_wage)
                best_option = min(costos, key=costos.get)
                best_cost = costos[best_option]

                if best_option == "Alquiler":
                    alquiler_estado[chacra] = 4
                else:
                    alquiler_contador[chacra] = 0

                if best_cost == 0:
                    best_option = "N/A"  # Asignar 'N/A' cuando el costo es cero



                exploded_results.append({
                    "Año": year,
                    "Semana": week,
                    "Chacra": chacra,
                    "Balance": balance,
                    "Ociosidad": costos["Ociosidad"],
                    "Sprinter": costos["Sprinter"],
                    "Bus": costos["Bus"],
                    "Vivienda Móvil": costos["Móvil"],
                    "Alquiler": costos["Alquiler"],
                    "Opcion": best_option,
                    "Costo Total": best_cost,
                })


# Convertir a DataFrame final
exploded_results_df = pd.DataFrame(exploded_results)

# Guardar y mostrar resultados
exploded_results_df.to_excel("Optimizacion_Logistica.xlsx", index=False)
display(exploded_results_df)

from google.colab import files
files.download("Optimizacion_Logistica.xlsx")


,Año,Semana,Chacra,Balance,Ociosidad,Sprinter,Bus,Vivienda Móvil,Alquiler,Opcion,Costo Total
0,Año 1,Semana 2,Blanquita,20.0,5500.0,4620.0,5500.0,625.0,inf,Móvil,625.0
1,Año 1,Semana 2,La Irene,6.0,1650.0,2310.0,5500.0,375.0,150.0,Alquiler,150.0
2,Año 1,Semana 2,La Carolina,1.0,275.0,2310.0,5500.0,375.0,150.0,Alquiler,150.0
3,Año 1,Semana 2,La Costa,1.0,275.0,2310.0,5500.0,375.0,500.0,Ociosidad,275.0
4,Año 1,Semana 2,El Mirasol,-4.0,inf,2310.0,5500.0,375.0,inf,Móvil,375.0
...,...,...,...,...,...,...,...,...,...,...,...
1185,Año 10,Semana 18,La Carolina,2.0,550.0,2310.0,5500.0,375.0,150.0,Alquiler,150.0
1186,Año 10,Semana 18,La Costa,-20.0,inf,4620.0,5500.0,625.0,500.0,Alquiler,500.0
1187,Año 10,Semana 18,El Mirasol,4.0,1100.0,2310.0,5500.0,375.0,inf,Móvil,375.0
1188,Año 10,Semana 18,San Victorio,19.0,5225.0,4620.0,5500.0,625.0,inf,Móvil,625.0


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>